## Monte Carlo - Black-Scholes-Merton

### We will work to calculate call option premium for Microsoft (MSFT), with risk free rate, r, equal to 2.5% (0.025); the strike price, K, equal to 110.0; and the time horizon, T, equal to 1 year, respectively.

### Download the data for Microsoft (‘MSFT’) from Yahoo Finance for the period ‘2000-1-1’ until today.

In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
from scipy.stats import norm

In [2]:
ticker = 'MSFT'  
data = pd.DataFrame()  
data[ticker] = wb.DataReader(ticker, data_source='yahoo', start='2000-1-1')['Adj Close']
data.head()

,MSFT
Date,
1999-12-31,37.162327
2000-01-03,37.102634
2000-01-04,35.849308
2000-01-05,36.227283
2000-01-06,35.013741


### We will understand certain functions to use ahead

### Standard Normal Distribution cdf

#### The cumulative distribution function (CDF) of a real-valued random variable X, evaluated at x, is the probability that X will take a value less than or equal to x. We can use the norm.cdf function from scipy module. 


In [3]:
norm.cdf(0) # In standard normal dist., the probability of variable equal to or less than 0 is 0.5 as it is the mean 

0.5

In [4]:
norm.cdf(0.25) # In standard normal dist., the probability of variable equal to or less than 0.25 is 0.598

0.5987063256829237

#### We will use norm cdf in Black-Scholes-Merton formula. 

### Black-Scholes-Merton
$$
\textbf{C} = SN(d_1) - Ke^{-rt}N(d_2) 
$$

where,


$$
d_1 = \frac{\ln(\frac{S}{K}) + (r + \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

$$
d_2 = d_1 - s \cdot \sqrt{t} = \frac{\ln(\frac{S}{K}) + (r - \frac{stdev^2}{2})t}{s \cdot \sqrt{t}}
$$

$$
C = Call Premium
$$
$$
K	=	strike price
$$
$$
r	=	risk free interest rate
$$
$$
t	=	time to maturity
$$
$$
s	=	asset volatility 
$$


### Calculating d1 and d2

In [5]:
def d1(S, K, r, stdev, T):
    return (np.log(S / K) + (r + stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))
 
def d2(S, K, r, stdev, T):
    return (np.log(S / K) + (r - stdev ** 2 / 2) * T) / (stdev * np.sqrt(T))


### Defining Black-Scholes-Merton formula using d1 and d2

In [18]:
def BSM(S, K, r, stdev, T):
        return (S * norm.cdf(d1(S, K, r, stdev, T))) - (K * np.exp(-r * T) * norm.cdf(d2(S, K, r, stdev, T)))

Store the annual standard deviation of the log returns in a variable, called “stdev”.

In [10]:
log_returns = np.log(1 + data.pct_change())
log_returns.head()

,MSFT
Date,
1999-12-31,NaN
2000-01-03,-0.001608
2000-01-04,-0.034364
2000-01-05,0.010488
2000-01-06,-0.034072


In [12]:
stdev = log_returns.std() * 250**0.5 # 250 for annual as there are approx. 250 trading sessions in an year
stdev

MSFT    0.308569
dtype: float64

Set the risk free rate, r, equal to 2.5% (0.025); the strike price, K, equal to 110.0; and the time horizon, T, equal to 1, respectively.

In [13]:
r = 0.025
K = 110.0
T = 1

Create a variable S equal to the last adjusted closing price of Microsoft. Use the “iloc” method.

In [14]:
S = data.iloc[-1]
S

MSFT    215.229996
Name: 2020-11-27 00:00:00, dtype: float64

Call the d1 and d2 functions with the relevant arguments to obtain their values.

In [15]:
d1(S, K, r, stdev, T)

MSFT    2.410592
dtype: float64

In [16]:
d2(S, K, r, stdev, T)

MSFT    2.102023
dtype: float64

Use the BSM function to estimate the price of a call option, given you know the values of S, K, r, stdev, and T.

In [21]:
C = BSM(S, K, r, stdev, T)

In [28]:
print(str("The call option premium for MSFT is: ")+str(C.values))

The call option premium for MSFT is: [108.13900136]
